In [7]:
import pandas as pd
import plotly.express as px

In [8]:
layout_options = {
    'paper_bgcolor':"#383838",
    'plot_bgcolor':'#383838',
    'title_font': dict(color='white'),
    'legend_font': dict(color='white'),
    'yaxis':dict(color="white"),
    'xaxis':dict(color="white")
    }

In [9]:
matches = pd.read_csv("data/matches_combined_data.csv")

### The mean/median of attendence by tournament

In [15]:
att_data = matches[["tournament_id","Attendance"]].groupby(
    "tournament_id"
    ).agg(
        median=('Attendance', 'median'),
        mean=('Attendance', 'mean')
    ).reset_index()

In [19]:
fig = px.line(
    att_data, 
    x="tournament_id",
    y=["median", "mean"],
    title="Median, Mean of Attendance by Tournament"
)
fig.update_layout(**layout_options)
fig.show()

الملاحظ من المخطط السابق أن لعبة كرة القدم تأخذ شعبية أكثر فاكثر بسبب ارتفاع معدلات الحضور بشكل مستمر في كل موسم
ويمكن أن يعزى ازدياد عدد المشاهدين عام 1994 لى عدة عوامل. منها: الشعبية المتزايدة لكرة القدم في الولايات المتحدة. بالإضافة إلى  انها النسخة الأولى التي تضم 24 فريقًا بدلاً من 16 ، مما زاد من عدد المباريات ومنح المزيد من الدول فرصة المشاركة.

----------------

### The histogram of attendence

In [20]:
fig = px.histogram(matches ,x="Attendance", nbins=100)
fig.update_layout(**layout_options)
fig.show()

---------------------------

### Attendece distribution using boxplot

In [13]:
fig = px.box(matches, x='tournament_id', y='Attendance', title='Attendance by Tournament')
fig.update_layout(**layout_options)
fig.show()

-----------------

In [65]:
goals = pd.read_csv("data/goals_data.csv")
players = pd.read_csv("data/players_data.csv")
player_appear = pd.read_csv("data/fjelstul/player_appearances.csv")


### The median of goals time

In [23]:
goal_period = goals[["tournament_id", "minute_regulation"]].groupby("tournament_id").median()

In [26]:
fig = px.bar(goal_period, y="minute_regulation", title="The median of Goals minute by tournament")
fig.update_layout(**layout_options)
fig.show()

-----------------------------

### A histogram for number of goals by match

In [45]:
fig = px.histogram(matches, x="total_goals_in_match")
fig.update_layout(**layout_options)
fig.show()

__________________

### The most frequent goal time

In [28]:
goals["minute_actual"] = goals["minute_regulation"] + goals["minute_stoppage"]
goals_min = goals[["tournament_id", "minute_actual"]].groupby("tournament_id").apply(lambda x: x.mode())

In [29]:
fig = px.bar(goals_min,x="tournament_id", y="minute_actual", range_y=[0, 120] )
fig.update_layout(**layout_options)
fig.show()

---------------------

### A histogram for late goals

In [32]:
goals_count = goals[["goal_id", "tournament_id"]][goals.late_goal == 1].groupby("tournament_id").count()


In [36]:
fig = px.histogram(goals_count, nbins=20)
fig.update_layout(**layout_options)
fig.show()

### the ratio of late goals to total goals 

In [37]:
late_goals_count = goals[["goal_id", "tournament_id"]][goals.late_goal == 1].groupby("tournament_id").count()
total_goals_count = goals[["goal_id", "tournament_id"]].groupby("tournament_id").count()

In [40]:
fig = px.bar(late_goals_count/total_goals_count)
fig.update_layout(**layout_options)
fig.show()

----------------------

### The bar chart for the best 12 scorers

In [69]:
player_goals = goals[["player_id", "goal_id"]].groupby("player_id").count().nlargest(12, columns=["goal_id"])

In [70]:
player_ids= player_goals.index.to_list()
player_names = players[["full_name", "player_id"]][players.player_id.isin(player_ids)]
player_goals = pd.merge(player_goals, player_names, on="player_id")

In [64]:
fig = px.bar(player_goals, x="full_name", y="goal_id")
fig.update_layout(**layout_options)
fig.show()

In [68]:
# # total goals / num matches for each player
# player_goals = goals[["player_id", "goal_id"]].groupby("player_id").count().nlargest(12, columns=["goal_id"])
# player_matches = player_appear[["player_id", "match_id"]].groupby("player_id").count().nlargest(12, columns=["match_id"])
# goals_mathes_ratio = player_goals["goal_id"] / player_matches["match_id"]
# goals_mathes_ratio 

---------------------

### The best scorer per tournament

In [80]:
player_max_goals = goals[["tournament_id", "player_id", "goal_id"]].groupby(
    ["tournament_id","player_id"]
).count().reset_index().groupby("tournament_id").max().reset_index()

In [81]:

player_ids= player_max_goals.player_id.to_list()
player_names = players[["full_name", "player_id"]][players.player_id.isin(player_ids)]
player_max_goals = pd.merge(player_max_goals, player_names, on="player_id")
player_max_goals["player_tournament"] = player_max_goals.full_name + " (" +player_max_goals.tournament_id + ")"

In [82]:
fig = px.bar(player_max_goals, x="player_tournament", y="goal_id")
fig.update_layout(**layout_options)
fig.show()

--------------------

### the total goals per tournament

In [84]:
goals_count = goals[["goal_id", "tournament_id"]].groupby("tournament_id").count()
fig = px.bar(goals_count,y="goal_id")
fig.update_layout(**layout_options)
fig.show()

In [77]:
import plotly.express as px

df = px.data.tips()
fig = px.strip(df, x=["total_bill"], y="day")
fig.show()